# Bert를 사용한 문장 간 관계 분류

2개의 문장을 주고, 3개의 카테고리 중의 하나로 분류한다.

연관, 중립, 상반

copy from https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/blob/main/7.PRETRAIN_METHOD/7.2.2.bert_finetune_KorNLI.ipynb

# 필요 라이브러리 설치

In [1]:
!pip install transformers==3.0.2
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 12.0 MB/s 
     |████████████████████████████████| 3.0 MB 51.5 MB/s 
     |████████████████████████████████| 880 kB 59.7 MB/s 
     |████████████████████████████████| 1.2 MB 65.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2b53c854d8caf45ae6d403b75da2f8cdda05c588239e65dca93e696fe4d3b3ca
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from transformers import BertTokenizer
from transformers import TFBertModel

import tensorflow as tf

In [3]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

SEQ_LENGTH = 128
BERT_MODEL_NAME = 'bert-base-multilingual-cased'

# 데이터

## 데이터 다운로드

In [4]:
!git clone https://github.com/kakaobrain/KorNLUDatasets

Cloning into 'KorNLUDatasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [5]:
!wc ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv

  550153  8595590 78486224 ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv


## 데이터 로딩

In [6]:
df = pd.read_csv("KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv", delimiter = '\t', quoting = 3)

In [7]:
df.head()

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment


## 카테고리 인덱스 만들기

In [8]:
df.gold_label = df.gold_label.astype('category')

In [9]:
df['category'] = df.gold_label.cat.codes

In [10]:
df.head()

,sentence1,sentence2,gold_label,category
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral,2
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction,0
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment,1
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral,2
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment,1


In [11]:
category_names = list(df.gold_label.cat.categories)
print(category_names)

['contradiction', 'entailment', 'neutral']


## 데이터 섞기

In [12]:
df = df.sample(frac=1).reset_index(drop=True) 

df.head()

,sentence1,sentence2,gold_label,category
0,다 큰 남자는 스파이더맨 침대보로 만든 옷을 입고 있다.,한 남자가 스파이더맨 침대보를 망토로 사용하여 슈퍼 스파이디가 될 수 있다!,neutral,2
1,회색 셔츠를 입은 남자가 사교 모임을 위해 사운드보드를 작동시키고 있다.,DJ가 음악을 연주하고 있다.,neutral,2
2,자전거를 탄 한 남자가 모터 크로스 행사 중에 흙을 토한다.,폭주족이 이벤트에서 이기고 있다.,neutral,2
3,한 남자가 배트맨 복장을 하고 포즈를 취하고 있다.,남자가 변장을 하고 있다.,entailment,1
4,깨끗한 컵 용기에 액체를 섞고 있는 한 여성과 함께 실험실처럼 보이는 대머리 남자,여자가 남자가 필로폰을 만드는 것을 보고 있다.,neutral,2


## 필요 입출력 값 준비

In [14]:
sentences1 = df.sentence1.values.copy().astype(str)
sentences2 = df.sentence2.values.copy().astype(str)
labels = df.category.values.copy().astype(np.int16)

In [ ]:
print(sentences1.shape)
print(sentences2.shape)
print(labels.shape)

(550152,)
(550152,)
(550152,)


필요 시, 실습 시간 관계로 전체 중에 일부 만 사용한다.

In [ ]:
COUNT = 10000
sentences1 = sentences1[:COUNT]
sentences2 = sentences2[:COUNT]
labels = labels[:COUNT]

## 토큰나이저 생성

In [ ]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_lower_case=False, model_max_length=SEQ_LENGTH)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

In [ ]:
encoded_tokens= tokenizer.encode("하늘이 푸르다.", text_pair="파란색이 좋아.")
print(encoded_tokens)
print(tokenizer.convert_ids_to_tokens(encoded_tokens))

[101, 9952, 118762, 10739, 9935, 31401, 11903, 119, 102, 9901, 49919, 41442, 10739, 9685, 16985, 119, 102]
['[CLS]', '하', '##늘', '##이', '푸', '##르', '##다', '.', '[SEP]', '파', '##란', '##색', '##이', '좋', '##아', '.', '[SEP]']


In [ ]:
tokenized = tokenizer("하늘이 푸르다.", text_pair="파란색이 좋아.", max_length=20, padding='max_length')
print(tokenizer.decode(tokenized['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized['input_ids']))
print(tokenized['input_ids'])
print(tokenized['attention_mask'])
print(tokenized['token_type_ids'])

[CLS] 하늘이 푸르다. [SEP] 파란색이 좋아. [SEP] [PAD] [PAD] [PAD]
['[CLS]', '하', '##늘', '##이', '푸', '##르', '##다', '.', '[SEP]', '파', '##란', '##색', '##이', '좋', '##아', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]']
[101, 9952, 118762, 10739, 9935, 31401, 11903, 119, 102, 9901, 49919, 41442, 10739, 9685, 16985, 119, 102, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]


## x, y 생성


tokernizer 사용 중에 경고 메시지가 많이 뜬다. 억제한다.


In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
def build_model_input(sentences1, sentences2):
  input_ids = []
  attention_masks = []
  token_type_ids = []

  for sentence1, sentence2 in zip(sentences1, sentences2):
    tokenized = tokenizer(sentence1, text_pair=sentence2, max_length=SEQ_LENGTH, padding='max_length')
    # tokenized = {'input_ids': [101, ...], 'token_type_ids': [0, ...], 'attention_mask': [1, ...]}
    input_ids.append(tokenized['input_ids'][:SEQ_LENGTH]) # 버그인지 몰라도 SEQ_LENGTH이상이어도 더 크게 나온다.
    attention_masks.append(tokenized['attention_mask'][:SEQ_LENGTH])
    token_type_ids.append(tokenized['token_type_ids'][:SEQ_LENGTH])

  return (np.array(input_ids), np.array(attention_masks), np.array(token_type_ids))


In [ ]:
x = build_model_input(sentences1, sentences2)
y = labels

In [ ]:
print(x[0].shape)

(10000, 128)


## train/test 분리

In [ ]:
def split_bert_data(x, y, test_ratio):
  split_index = int(len(y)*(1-test_ratio))
  train_x = (x[0][:split_index], x[1][:split_index], x[2][:split_index])
  test_x  = (x[0][split_index:], x[1][split_index:], x[2][split_index:])
  train_y, test_y = y[:split_index], y[split_index:]

  return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = split_bert_data(x, y, test_ratio=0.2)

# 학습

## 모델 생성

In [ ]:
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Dense, Dropout

class TFBertClassifier(tf.keras.Model):
  def __init__(self):
    super(TFBertClassifier, self).__init__()

    self.bert = TFBertModel.from_pretrained(BERT_MODEL_NAME)
    self.dropout = Dropout(self.bert.config.hidden_dropout_prob)
    self.classifier = Dense(3, kernel_initializer=TruncatedNormal(self.bert.config.initializer_range), 
                            name="classifier", activation="softmax")

  def call(self, inputs, attention_mask=None, token_type_ids=None, training=True):

    outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
    pooled_output = outputs[1] 
    v = self.dropout(pooled_output, training=training)
    out = self.classifier(v)

    return out

model = TFBertClassifier()


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

참고로 Bert의 default 설정은 다음과 같다.

In [ ]:
print(model.bert.config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}



In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

optimizer = Adam(3e-5)
loss = SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


## 학습 실행

In [ ]:
history = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
225/225 [==============================] - 73s 278ms/step - loss: 0.9200 - accuracy: 0.5661 - val_loss: 0.7556 - val_accuracy: 0.6725
Epoch 2/5
225/225 [==============================] - 61s 271ms/step - loss: 0.6807 - accuracy: 0.7210 - val_loss: 0.7091 - val_accuracy: 0.7175
Epoch 3/5
225/225 [==============================] - 61s 271ms/step - loss: 0.5215 - accuracy: 0.8003 - val_loss: 0.7505 - val_accuracy: 0.6963
Epoch 4/5
225/225 [==============================] - 61s 272ms/step - loss: 0.3767 - accuracy: 0.8593 - val_loss: 0.8612 - val_accuracy: 0.7075
Epoch 5/5
225/225 [==============================] - 61s 272ms/step - loss: 0.2760 - accuracy: 0.9004 - val_loss: 0.9142 - val_accuracy: 0.6925


In [ ]:
loss, acc = model.evaluate(test_x, test_y, batch_size=32)
print("loss =", loss)
print("acc =", acc)

63/63 [==============================] - 9s 99ms/step - loss: 0.9221 - accuracy: 0.6945
loss = 0.9220576286315918
acc = 0.6945000290870667


## 분류 실행

In [ ]:
def do_classify(sentence1, sentence2):
  model_input = build_model_input([sentence1], [sentence2])
  y_ = model.predict(model_input)
  predicted = np.argmax(y_, axis=-1)[0]
  print(sentence1, sentence2, "-->", category_names[predicted], ",score :",y_[0][predicted])

do_classify("나는 왜 그런지 잘 모르겠다.", "나는 그 이유에 관해 확신한다.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 그가 왜 학교를 전학했는지 모르겠다.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 왜 그런 일이 일어났는지 모르겠어.")

나는 왜 그런지 잘 모르겠다. 나는 그 이유에 관해 확신한다. --> neutral ,score : 0.647574
나는 왜 그런지 잘 모르겠다. 나는 그가 왜 학교를 전학했는지 모르겠다. --> neutral ,score : 0.98947644
나는 왜 그런지 잘 모르겠다. 나는 왜 그런 일이 일어났는지 모르겠어. --> neutral ,score : 0.95380366
